In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop, ColorJitter
from PIL import Image

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import os

In [2]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'expanded_train.csv')
dt_train

,Unnamed: 0,PersonID,Filename,Class,Mask,Gender,Age,Age_Class,Has_Face,BBoxX1,...,LE_X,LE_Y,RE_X,RE_Y,N_X,N_Y,LM_X,LM_Y,RM_X,RM_Y
0,0,000001,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,True,112,...,145,230,206,230,176,268,158,296,198,297
1,1,000001,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,True,120,...,150,223,210,219,182,257,165,289,203,287
2,2,000001,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,True,107,...,134,205,193,206,162,243,145,274,184,274
3,3,000001,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,True,110,...,139,232,209,234,171,275,151,307,194,309
4,4,000001,../input/data/train/images/000001_female_Asian...,10,1,1,45,1,True,122,...,152,205,211,202,182,245,164,275,205,273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,18895,006959,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,True,116,...,149,235,212,234,180,273,161,301,202,300
18896,18896,006959,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,True,120,...,154,234,223,235,186,273,164,302,211,303
18897,18897,006959,../input/data/train/images/006959_male_Asian_1...,6,1,0,19,0,True,111,...,144,241,206,241,173,273,153,306,195,307
18898,18898,006959,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,True,112,...,151,233,226,237,184,279,160,311,209,314


In [4]:
#whole_image_path = []
#whole_target_label = []

#for path in dt_train['path']:
#    for file_name in [i for i in os.listdir(train_image_path+path) if './' not in i]:
#        whole_image_path.append(train_image_path+path+'/'+file_name)
#        whole_target_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))

whole_image_path = dt_train.loc[:,"Filename"]
whole_target_label = dt_train.loc[:,"Class"]

~~
def onehot_enc(x):
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])
~~

In [5]:
sr_data = pd.Series(whole_image_path)
sr_label = pd.Series(whole_target_label)

In [6]:
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop, ColorJitter
from PIL import Image

class Dataset_Mask(Dataset):
    def __init__(self, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = sr_data
        self.label = sr_label
        self.transform = transform
        
        #if encoding:
        #    self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(sr_data)
    
    def __getitem__(self, idx):
        #X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        X = np.load(self.data[idx])
        y = self.label[idx]
        
        if self.midcrop:
            # X = X[64:448]
            x1 = dt_train.iloc[idx]['BBoxX1']
            y1 = dt_train.iloc[idx]['BBoxY1']
            x2 = dt_train.iloc[idx]['BBoxX2']
            y2 = dt_train.iloc[idx]['BBoxY2']
        X = cv2.resize(X, (224, 224))
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [7]:
dataset_mask = Dataset_Mask(transform = transforms.Compose([
                                    ToTensor(),
                                    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                            ]))

In [8]:
train_size = int(len(dataset_mask) * 0.8)
val_size = len(dataset_mask) - train_size

In [9]:
mask_train_set, mask_val_set = torch.utils.data.random_split(dataset_mask, [train_size, val_size])
print(f'training data size : {len(mask_train_set)}')
print(f'validation data size : {len(mask_val_set)}')

training data size : 15120
validation data size : 3780


In [10]:
batch_size = 256
train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [11]:
basemodel_resnet34 = torchvision.models.resnet34(pretrained=True)
print('필요 입력 채널 개수', basemodel_resnet34.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', basemodel_resnet34.fc.weight.shape[0])
print(basemodel_resnet34)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /opt/ml/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth



필요 입력 채널 개수 3
네트워크 출력 채널 개수 1000
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [12]:
import math
class_num = 18
basemodel_resnet34.fc = nn.Linear(in_features=512, out_features=class_num, bias=True)
nn.init.xavier_uniform_(basemodel_resnet34.fc.weight)
stdv = 1. / math.sqrt(basemodel_resnet34.fc.weight.size(1))
basemodel_resnet34.fc.bias.data.uniform_(-stdv, stdv)

print('필요 입력 채널 개수', basemodel_resnet34.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', basemodel_resnet34.fc.weight.shape[0])

필요 입력 채널 개수 3
네트워크 출력 채널 개수 18


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

basemodel_resnet34.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 30

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(basemodel_resnet34.parameters(), lr=LEARNING_RATE)

using cuda:0


In [14]:
np.set_printoptions(precision=3)
n_param = 0
for p_idx, (param_name, param) in enumerate(basemodel_resnet34.named_parameters()):
    if param.requires_grad:
        param_numpy = param.detach().cpu().numpy()
        n_param += len(param_numpy.reshape(-1))
        print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
        print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
print ("Total number of parameters:[%s]."%(format(n_param,',d')))

[0] name:[conv1.weight] shape:[(64, 3, 7, 7)].
    val:[ 0.005 -0.007  0.008  0.038  0.049]
[1] name:[bn1.weight] shape:[(64,)].
    val:[0.302 0.268 0.26  0.311 0.238]
[2] name:[bn1.bias] shape:[(64,)].
    val:[0.481 0.207 0.331 0.38  0.094]
[3] name:[layer1.0.conv1.weight] shape:[(64, 64, 3, 3)].
    val:[-0.005  0.015 -0.006 -0.06  -0.024]
[4] name:[layer1.0.bn1.weight] shape:[(64,)].
    val:[0.24  0.185 0.216 0.165 0.181]
[5] name:[layer1.0.bn1.bias] shape:[(64,)].
    val:[0.025 0.088 0.082 0.142 0.066]
[6] name:[layer1.0.conv2.weight] shape:[(64, 64, 3, 3)].
    val:[ 0.066 -0.01   0.041  0.033 -0.055]
[7] name:[layer1.0.bn2.weight] shape:[(64,)].
    val:[0.34  0.187 0.252 0.307 0.259]
[8] name:[layer1.0.bn2.bias] shape:[(64,)].
    val:[-0.251  0.196  0.23  -0.114  0.07 ]
[9] name:[layer1.1.conv1.weight] shape:[(64, 64, 3, 3)].
    val:[-0.008 -0.04  -0.054 -0.019  0.011]
[10] name:[layer1.1.bn1.weight] shape:[(64,)].
    val:[0.178 0.373 0.18  0.26  0.246]
[11] name:[layer1.

In [15]:
best_val_acc = 0
best_val_loss = np.inf

for epoch in range(NUM_EPOCH):
    basemodel_resnet34.train()
    loss_value = 0
    matches = 0
    for train_batch in train_dataloader_mask:
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = basemodel_resnet34(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
        train_loss = loss_value / batch_size
        train_acc = matches / batch_size
        print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.3f}, training accuracy {train_acc:.3f}")
        
        loss_value = 0
        matches = 0
        
    with torch.no_grad():
        basemodel_resnet34.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in val_dataloader_mask:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = basemodel_resnet34(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            
        print(f"[val] acc : {val_acc:.3f}, loss : {val_loss:.3f}")
        print(f"best acc : {best_val_acc:.3f}, best loss : {best_val_loss:.3f}")

epoch[0/30] training loss 0.016, training accuracy 0.035
epoch[0/30] training loss 0.012, training accuracy 0.137
epoch[0/30] training loss 0.009, training accuracy 0.336
epoch[0/30] training loss 0.008, training accuracy 0.441
epoch[0/30] training loss 0.007, training accuracy 0.574
epoch[0/30] training loss 0.006, training accuracy 0.629
epoch[0/30] training loss 0.005, training accuracy 0.660
epoch[0/30] training loss 0.004, training accuracy 0.746
epoch[0/30] training loss 0.003, training accuracy 0.730
epoch[0/30] training loss 0.004, training accuracy 0.711
epoch[0/30] training loss 0.003, training accuracy 0.805
epoch[0/30] training loss 0.003, training accuracy 0.777
epoch[0/30] training loss 0.003, training accuracy 0.727
epoch[0/30] training loss 0.002, training accuracy 0.824
epoch[0/30] training loss 0.003, training accuracy 0.797
epoch[0/30] training loss 0.002, training accuracy 0.805
epoch[0/30] training loss 0.002, training accuracy 0.852
epoch[0/30] training loss 0.002

KeyboardInterrupt: 

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

In [ ]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [ ]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            img = img[64:448]
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [ ]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToTensor()
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = basemodel_resnet34.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission.csv'), index=False)
print('test inference is done!')